<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P3: Visualizing a large Dataset and Checking for Simpsons Paradox
Below you will find a large(r) dataset with time series data of environmental sensors


Hint: Rooms with a '0' as second digit have a different orientation than rooms with a '1'

You might use you own dataset as well under the asumption:
* you have time series data
* you have at least three additional features that depend on time


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import plotly.express as px
import plotly.graph_objects as go

## 1 Load the data into a dataframe and convert the date strings

Hint, you might also find a parquet file. In that case, you can use the parquet file. It has the date already converted.

In [ ]:
%time df = pd.read_csv(r"..\07092021bimsim_room.csv", sep=",")

## 2 Remove duplicate or unused data to reduce total memory consumption

Domain knowledge:
* the columns "source" and "sensor" are identical
* "sensor" and "room" have a 1:1 mapping
* "id" contains the datetime as float
* some sensors might deliver negative temperatures <100°C. This is an errorneous measurement

In [ ]:
# your code here

## 3 Visualize the data 

Perform Visualizations for different types of diagrams using either

1. matpolotlib/seaborn
2. plotly

**Important Hint**: Check the total volume of data points first. If needed create a digestible subset. Keep in mind, that visualizing large volumes of data points (i.e. > 10e5) can significantly slow down the notebook. Too many datapoints might even crash the browser, due to limited RAM availability for a browser tab in all common browsers.

## 3.0 Create different table aggregations to check for "Simposons Paradox"
Hints: 
* Find an appropriate aggregation of which you think, it could generate a paradox. E.g. compare an aggregation of multiple rooms with a single rooom.
* Rooms with a '0' in the middle are on one side of the building, with a '1' on another.
* There are also errorneous values in the data, that you might want to remove first.
* Consider different time spans. During the winter period, many rooms a heated. During summer, the room temperatures are more influenced by the outside environmental conditions

In [ ]:
# your code here

## 3.1 Create a graph over time
Use the line or scatter graphs to check for simpsons paradox.
What is the difference in the perception of the data when using line graphs compared to scatter graphs? Think about missing data!

In [ ]:
# your code here

In [ ]:
# your code here

## 3.2 Create a 2D and 3D scatterplot for the environmental data

In [ ]:
# your code here